In [2]:
# importar librerias
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

In [3]:
### para graficar localmente
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## Balance energetico

In [4]:
# cargar datos
df_balance = pd.read_csv('insumos/electricidad-balance_1992-2018_anual.csv', sep=";")

In [15]:
df_balance.head()

,anio,variable,valor_GWh
0,1992,demanda_agentes_mem,49714.6
1,1993,demanda_agentes_mem,52660.4
2,1994,demanda_agentes_mem,55995.0
3,1995,demanda_agentes_mem,58012.2
4,1996,demanda_agentes_mem,62018.0


In [16]:
df_balance.variable.value_counts()

demanda_exportacion          27
oferta_eolica_solar          27
oferta_importacion           27
oferta_hidraulica            27
racionamiento_cortes         27
demanda_agentes_mem          27
racionamiento_tension        27
oferta_total                 27
oferta_nuclear               27
demanda_bombeo               27
requerido_total              27
demanda_total                27
oferta_termica               27
demanda_perdidas_consumos    27
Name: variable, dtype: int64

Dado que los valores son anuales, saco el dato 2019 que solo incluye los datos del primer trimestre.

In [13]:
df_balance.drop(df_balance[df_balance.anio == 2019].index, inplace=True)

Calculo porcentaje del consumo eléctrico que se cubre con importaciones de electricidad. __Faltaría ver porcentaje de la producción eléctrica que se hace con combustibles importados__

In [23]:
porcentaje_importado = (df_balance.query('variable == "oferta_importacion"').valor_GWh.values / 
                       df_balance.query('variable == "oferta_total"').valor_GWh.values * 100)
porcentaje_importado

array([4.19395419, 2.07363101, 0.53460786, 0.47923002, 0.40435893,
       0.60726083, 2.50229769, 0.40855811, 1.20716418, 1.68590145,
       2.71717748, 1.42754019, 1.54471362, 1.24490501, 0.53427891,
       3.18854741, 1.57871976, 1.83233917, 2.03137353, 1.98973704,
       0.33607728, 0.26367359, 1.05910679, 1.20899829, 1.06460799,
       0.53491799, 0.24922891])

In [29]:
balance_demanda = go.Scatter(
    x=df_balance.anio,
    y=df_balance.query('variable == "demanda_total"').valor_GWh,
    name='Demanda Energética'
)

balance_oferta = go.Bar(
    x=df_balance.anio,
    y=porcentaje_importado,
    yaxis='y2',
    name='% Importado'
)
data = [balance_demanda, balance_oferta]
layout = go.Layout(
    title='Matriz Energética: Demanda y Oferta',
    yaxis=dict(
        title='Consumo Eléctrico (GWh)',
        range=[0, 150000],
    ),
    yaxis2=dict(
        title='Importación (% del total)',
        overlaying='y',
        side='right',
        range=[0, 10]
    ),
    xaxis=dict(
        tickmode='linear',
        ticks='inside',
        tick0=1995,
        dtick=5
    ),
    legend=dict(orientation="h")
)
fig = go.Figure(data=data, layout=layout)

# configurar para que se vea bien en celulares
fig['layout'].update(height=300, width=400)
fig['layout']['margin'].update(l=25, r=25, b=50, t=50, pad=0)
fig['layout']['yaxis'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis2'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['title'].update(font=dict(size=12))

# visualizar localmente
iplot(fig, filename='balance-energetico')

# publicar
#py.iplot(fig, filename='produccion-hidrocarburos-anual', sharing='public')